In [1]:
try:
  # We must install required packages if we are in Google Colab
  import google.colab
  %pip install roboticstoolbox-python>=1.0.2
except:
  # We are not in Google Colab
  # Apply custon style to notebook
  from IPython.core.display import HTML
  import pathlib
  styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
  styles = open(styles_path, "r").read()
  HTML(f"<style>{styles}</style>")

# 5.0 Manipulator Performance Measures

$\large{\text{Manipulator Differential Kinematics}} \\ \large{\text{Part I: Kinematics, Velocity, and Applications}}$

$\text{By Jesse Haviland and Peter Corke}$


<br>

The sections of the Tutorial paper related to this notebook are listed next to each contents entry.
It is beneficial to read these sections of the paper before attempting the notebook Section.

### Contents

[5.1 Manipulability Measure](#m)
* Manipulator Performance Metrics

[5.2 Condition Number](#cond)
* Manipulator Performance Metrics


In [2]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

Manipulator performance metrics seek to quantify the performance of a manipulator in a given configuration. In this section, we explore two common manipulator performance metrics based on the manipulator Jacobian.

Things to note
* The metrics are unitless, and the upper bound of a metric depends on the manipulator kinematic model (i.e. joint types and link lengths). Consequently, metrics computed for different manipulators are not directly comparable.
* The manipulator Jacobian contains three rows corresponding to translational rates, and three rows corresponding to angular rates. Therefore, any metric using the whole Jacobian will produce a non-homogeneous result due to the mixed units. Depending on the manipulator scale, this can cause either the translational or rotational component to dominate the result. This problem also arises in manipulators with mixed prismatic and revolute joints. In general, the most intuitive use of performance metrics comes from using only the translational or rotational rows of the manipulator Jacobian (where the choice of which depends on the use case), and only using the metric on a manipulator comprising with a single joint type.

<br>

<a id='m'></a>

### 5.1 Manipulability Measure
---

The Yoshikawa manipulability index is the most widely used and accepted performance metric. The index is calculated as

\begin{align*}
    m(\bf{q}) = \sqrt{
        \text{det}
        \Big(
            \hat{\bf{J}}(\bf{q})
            \hat{\bf{J}}(\bf{q})^\top
        \Big)
    }
\end{align*}

where $\hat{\bf{J}}(\bf{q}) \in \mathbb{R}^{3 \times n}$ is either the translational or rotational rows of $\bf{J}(\bf{q})$ causing  $m(\bf{q})$ to describe the corresponding component of manipulability. 
Note that Yoshikawa used $\bf{J}(\bf{q})$ instead of $\hat{\bf{J}}(\bf{q})$ in his original paper but we describe it so due to limitations of Jacobian based measures discussed above.
The scalar $m(\bf{q})$ describes the volume of a 3-dimensional ellipsoid -- if this ellipsoid is close to spherical, then the manipulator can achieve any arbitrary end-effector (translational or rotational depending on $\hat{\bf{J}}(\vec{q})$) velocity. The ellipsoid is described by three radii aligned with its principal axes. A small radius indicates the robot's inability to achieve a velocity in the corresponding direction. At a singularity, the ellipsoid's radius becomes zero along the corresponding axis and the volume becomes zero. If the manipulator's configuration is well conditioned, these ellipsoids will have a larger volume. Therefore, the manipulability index is essentially a measure of how easily a manipulator can achieve an arbitrary velocity.

We first make a method to calculate the manipulability of a robot using the above description.

In [3]:
def manipulability(robot, q, axes):
    """
    Calculates the manipulability of the robot at joint configuration q

    :robot: A Robot object to find the manipulability of
    :q: The joint coordinates of the robot (ndarray)
    :axes: A boolean list which correspond with the Cartesian axes to
        find the manipulability of (6 boolean values in a list)
    """
    # calculate the Jacobian
    J = robot.jacobe(q)

    # only keep the selected axes of J
    J = J[axes, :]

    # calculate the manipulability
    m = np.sqrt(np.linalg.det(J @ J.T))

    return m

We make a robot and define some axes selections. The lists `trans_axes`, `rot_axes`, `all_axes` each correspond to a set of Cartesian axes.

In [4]:
# Make a Panda
panda = rtb.models.Panda()

# Only the translation axes
trans_axes = [True, True, True, False, False, False]

# Only the rotation aces
rot_axes = [False, False, False, True, True, True]

# All axes
all_axes = [True, True, True, True, True, True]

Lets test out our method using the Panda robot in the ready `panda.qr` configuration.

In [5]:
print(f"Translational manipulability: {manipulability(panda, panda.qr, trans_axes)}")
print(f"Rotational manipulability: {manipulability(panda, panda.qr, rot_axes)}")
print(f"All axes manipulability: {manipulability(panda, panda.qr, all_axes)}")

Translational manipulability: 0.14384031993097537
Rotational manipulability: 2.745582183072283
All axes manipulability: 0.08375150968113343


In the `panda.qr` configuration, the Panda is well configured. That is to say that it is in a manipulable configuration and can easily move in any direction.

For the Panda, 0.14 is around the peak of the translational manipulability and 2.74 is in the upper range of the rotational manipulability. These two numbers are not comparable to each other due to being made from different units. We also can not compare these numbers to translational/rotational manipulabilities on other robots.

We can also see that the rotational manipulability is an order of magnitude greater than the translational manipulability. This difference can cause unintuitive results in the all axes manipulability measure.

In the next cell, we can see a test the manipulability of the Panda in a poorly configured pose `panda.qz`. This is the zero joint angle configuration in which the Panda is in a singularity.

In [6]:
print(f"Translational manipulability: {manipulability(panda, panda.qz, trans_axes)}")
print(f"Rotational manipulability: {manipulability(panda, panda.qz, rot_axes)}")
print(f"All axes manipulability: {manipulability(panda, panda.qz, all_axes)}")

Translational manipulability: 0.009904977685365267
Rotational manipulability: nan
All axes manipulability: nan


/var/folders/jm/65py5sf558zb166cykvzz9zw0000gq/T/ipykernel_50710/2288985554.py:17: RuntimeWarning: invalid value encountered in sqrt
  m = np.sqrt(np.linalg.det(J @ J.T))


<br>

<a id='cond'></a>

### 5.2 Condition Number
---

The condition number of the manipulator Jacobian was proposed as a performance measure. The condition number is

\begin{align*}
    \kappa =
    \dfrac{\sigma_{\text{max}}}
    {\sigma_{\text{min}}}
    \in [1, \ \infty ]
\end{align*}

where $\sigma_{\text{max}}$ and $\sigma_{\text{min}}$ are the maximum and minimum singular values of $\hat{\bf{J}}(\bf{q})$ respectively. The condition number is a measure of velocity isotropy. A condition number close to $1$ means that the manipulator can achieve a velocity in a direction equally as easily as any other direction. However, a high condition number does not guarantee a high manipulability index where the manipulator may struggle to move in all directions.

We first make a method to calculate the condition number of a robot using the above description.

In [7]:
def cond(robot, q, axes):
    """
    Calculates the condition number of the robot's Jacobian at
    joint configuration q

    :robot: A Robot object to find the manipulability of
    :q: The joint coordinates of the robot (ndarray)
    :axes: A boolean list which correspond with the Cartesian axes to
        find the manipulability of (6 boolean values in a list)
    """
    # calculate the Jacobian
    J = robot.jacobe(q)

    # only keep the selected axes of J
    J = J[axes, :]

    _, s, _ = np.linalg.svd(J)

    Jcond = np.max(s) / np.min(s)

    ## Note: we could just use the cond method from numpy
    # Jcond = np.linalg.cond(J)

    return Jcond

Lets test out our method using the Panda robot in the ready `panda.qr` configuration.

In [8]:
print(f"Translational condition number: {cond(panda, panda.qr, trans_axes)}")
print(f"Rotational condition number: {cond(panda, panda.qr, rot_axes)}")
print(f"All axes condition number: {cond(panda, panda.qr, all_axes)}")

Translational condition number: 2.5512647717700543
Rotational condition number: 2.031042407741737
All axes condition number: 8.910974536808437


Lets test out our method using the Panda robot in the ready `panda.qz` configuration.

In [9]:
print(f"Translational condition number: {cond(panda, panda.qz, trans_axes)}")
print(f"Rotational condition number: {cond(panda, panda.qz, rot_axes)}")
print(f"All axes condition number: {cond(panda, panda.qz, all_axes)}")

Translational condition number: 4.866993326664893
Rotational condition number: 1.0400617828738614e+16
All axes condition number: 6.968343580985044e+16


Remember that a condition number of 1 is perfect and infinity is the worst.